In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lynn310","key":"208beca6974a5ea1aa9fd1c4daea075c"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!pip install kaggle

!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c titanic

    100% |████████████████████████████████| 61kB 3.9MB/s 
    100% |████████████████████████████████| 235kB 7.6MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 21.4MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 26.9MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 4.24MB/s]


In [0]:
import numpy as np
import pandas as pd

# tools
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn import model_selection

import xgboost as xgb



import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [0]:
train = pd.read_csv("./train.csv", sep=",")
test = pd.read_csv("./test.csv", sep=",")

In [0]:
data = pd.concat([train, test])
train_size = train.shape[0] # 891
test_size = test.shape[0] # 418

In [0]:
data['Fare'] = data['Fare'].fillna(13.30)
# data = data.assign(
#     Fare = data.apply(lambda f: f.Fare if pd.notnull(f.Fare) else data[data['Pclass'] == f.Pclass]['Fare'].mean(), axis=1)
# )
data['Embarked'] = data['Embarked'].fillna('S')

In [0]:
data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=False)
# pd.crosstab(data['Title'], data['Pclass'])

In [0]:
age_ref = data.groupby('Title').Age.mean()
data = data.assign(
    Age = data.apply(lambda r: r.Age if pd.notnull(r.Age) else age_ref[r.Title] , axis=1)
)
del age_ref

In [0]:
data['Title'] = data['Title'].replace(['Don', 'Capt', 'Col', 'Major', 'Sir', 'Jonkheer', 'Rev', 'Dr'], 'Honored')
data['Title'] = data['Title'].replace(['Lady', 'Dona', 'Mme', 'Countess'], 'Mrs')
data['Title'] = data['Title'].replace(['Mlle', 'Ms'], 'Miss')
# data[['Title', 'Survived']].groupby(['Title']).agg(['count','mean'])

In [0]:
data['FamilySize'] = (data['SibSp'] + data['Parch'] + 1).astype(int)
# data[['FamilySize', 'Survived']].groupby(['FamilySize']).agg(['count','mean'])

In [0]:
cols = [
    'Pclass',
    'Age',
    'Sex',
    'FamilySize',
#     'SibSp',
#     'Parch',
    'Fare',
#     'Title',
    'Embarked'
]
X_train = data[:train_size][cols]
Y_train = data[:train_size]['Survived'].astype(int)
X_test = data[train_size:][cols]

# print(X_train.shape, Y_train.shape, X_test.shape)
# X_train.head()

In [0]:
one_hot_features = [
    'Pclass',
    'Sex',    
#     'Title',
    'Embarked'
]
X_train = pd.get_dummies(X_train, columns = one_hot_features)
X_test = pd.get_dummies(X_test, columns = one_hot_features)

# print(X_train.shape, Y_train.shape, X_test.shape)

In [0]:
xg_boost = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.65, gamma=2, learning_rate=0.3, max_delta_step=1,
       max_depth=4, min_child_weight=2, missing=None, n_estimators=280,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [154]:
xg_boost.fit(X_train, Y_train)
Y_pred = xg_boost.predict(X_test)
print(xg_boost.score(X_train, Y_train))

scores = model_selection.cross_val_score(xg_boost, X_train, Y_train, cv=5, scoring='accuracy')
print(scores)
print("Kfold on XGBClassifier: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std()))

0.8787878787878788
[0.81005587 0.82122905 0.84269663 0.79775281 0.85310734]
Kfold on XGBClassifier: 0.8250 (+/- 0.0204)


In [155]:
from sklearn.ensemble import GradientBoostingClassifier
forest = GradientBoostingClassifier(n_estimators=55, random_state=8)
forest = forest.fit(X_train, Y_train)
Y_pred_forest = forest.predict(X_test)

print(forest.score(X_train, Y_train))

scores = model_selection.cross_val_score(forest, X_train, Y_train, cv=5, scoring='accuracy')
print(scores)
print("Kfold on XGBClassifier: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std()))

0.8720538720538721
[0.81564246 0.82122905 0.8258427  0.80337079 0.84745763]
Kfold on XGBClassifier: 0.8227 (+/- 0.0145)


In [156]:
feature_importance = list(zip(X_train.columns.values, xg_boost.feature_importances_))
feature_importance.sort(key=lambda x:x[1])
feature_importance

[('Embarked_Q', 0.0),
 ('Embarked_C', 0.022058824),
 ('Pclass_2', 0.029411765),
 ('Sex_male', 0.029411765),
 ('Pclass_1', 0.036764707),
 ('Embarked_S', 0.036764707),
 ('Pclass_3', 0.05882353),
 ('Sex_female', 0.080882356),
 ('FamilySize', 0.11764706),
 ('Age', 0.2720588),
 ('Fare', 0.31617647)]

In [0]:
submission = pd.DataFrame({
    "PassengerId": data[train_size:]["PassengerId"], 
    "Survived": np.logical_and(Y_pred, Y_pred_forest)*1
})
submission.to_csv('submission.csv', index=False)